# Generative Adversarial Networks (GAN)

Adversarial Learning

* Adversarial machine learning is a technique used in machine learning (ML) to fool or misguide a model with malicious input.

* It includes both the generation and detection of adversarial examples, which are inputs specially created to deceive classifiers.



Generative Models

* Models that can generate new data samples from the learned data distribution.

* If we focus on images, then generative models learn the pixel intensity distribution. 


### [GAN](https://arxiv.org/abs/1406.2661)

* GAN is the **GENERATIVE**, **ADVERSARIAL** **UNSUPERVISED** learning algorithm

* GAN consists of two different AI models viz. Generator Network and Discriminator Network.

* Generator and Discriminator Networks compete against each other hence following adversarial learning.

* Working Mechanism Intuition:
    * Generator generates new data samples. A well trained generator generates realistic data (images in our case).

    * Discriminator is a classifier that classifes whether the input data(image) is a real one(i.e. images from training set) or a fake one (i.e. generated by generator).
    
    * Good discriminator can classify real and fake samples with good accuracy.

    * Good generator can create realistic fake samples and can fool discriminator.

    


## GAN Architecture
![gan-arch](./assets/gan-arch.png)

## Let's Build GAN by Ourselves

* Building GANs mean building two neural networks.

In [ ]:
# cell-1
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

### Data Preparation
* We will be using MNIST dataset which can be easily downloaded from torchvision-datasets.

* We will be using only images from that dataset not labels (unsupervised learning).

In [ ]:
# cell-2
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Visualization of MNIST Dataset Samples

In [ ]:
# cell-3
def imshow(img):
    img = img / 2 + 0.5  # Denormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

def show_grid(images):
    img_grid = torchvision.utils.make_grid(images, nrow=8)
    imshow(img_grid)

# Get some random training images
trainloader_for_vis = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
dataiter = iter(trainloader_for_vis)
images, labels = next(dataiter)

# Display the images in a grid
show_grid(images)

print(f"Shape of a batch of images: {images.shape}")

### Building Discriminator Network

* Discriminator is a classfier.

* Input: Image

* Ouput: Real or Fake

* We will be building a simple discriminator using Mulit-layer Perceptron.

* Do not forget to flatten the image since we will be using linear layers.

* Keep the architecture as simple as possible.

In [ ]:
# cell-4
# Your code here
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.flatten = nn.Flatten()
        
        self.classifier = nn.Sequential(
            ?
        )
    
    def forward(self, x):
        x = ?
        x = ?
        return x

### Building Generator Network

* We will be building simple generator network which is also a multi-layer perceptron network.

* Input to the generator will be a vector which is known as latent vector or noise vector.

* The dimension of latent vector is a hyperparameter. Let's use latent vector of dimension 100.

* Latent vector is usually sampled from normal distribution.

* Output of the generator will be a vector of dimension equal to training image dimension.

* Do not forget to reshape the output vector to 2d-image shape same as training image dimension.

In [ ]:
# cell-5
# Your code here

class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.mlp = nn.Sequential(
            ?
        )
    
    def forward(self, x):
        x = ?
        x = x.reshape(x.size(0), 1, 28, 28)
        return x

### Objective Function for GAN

**Discriminator Objective Function**

* Discriminator objective is to classify real image as 1 and fake image as 0.

* Discriminator is trained with both real images and fake images.

* Discriminator loss consists of two parts:

    1. real_data_loss: The loss when the discriminator correctly identifies real data as real.

    2. fake_data_loss: The loss when the discriminator correctly identifies fake data as fake.

* Since discriminator formulates binary classification problem, both real_data_loss and fake_data_loss are BinaryCrossEntropy Loss.

* Total discriminator loss = real_data_loss + fake_data_loss

* Objective: **Minimize(real_data_loss + fake_data_loss)**


**Generator Objective Function**
* Generator objective is to generate images that looks realistic. In other words, generator objective is to fool discriminator. In other words, generator objective is to increase discriminator loss and decrease discriminator accuracy.

* If the image generated by generator is classified as fake by discriminator, then generator_loss should increase. (In this scenario, generator fails to achieve its objective of fooling discriminator)

* If the image generated by generator is classified as real by discriminator, then generator_loss should decrease. (In this scenario, generator succeed to achieve its objective of fooling discriminator)

* We use BinaryCrossEntropy as loss for generator.

### Training GAN

* Training GAN involve training Discriminator and Generator alternatively.

In [ ]:
# cell-6
# Using available hardware device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating the generator and discriminator
latent_dim = 100
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# creating binary cross entropy loss instancea
criterion = nn.BCELoss()

# creating optimizers for generator and discriminator
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

In [ ]:
# cell-7
# Containers to store losses for visualization purpose
num_epochs = 20
discriminator_loss = []
generator_loss = []

### Steps for Training Discriminator

1. Infer discriminator with a mini-batch of real images from training set.
2. Calculate loss for discriminator with real data.
3. Generate latent vector from normal distribution for mini-batch.
4. Generate fake image by feeding latent vector to generator.
5. Infer discriminator with fake images.
6. Calulate loss for discriminator with fake data.
7. Compute total loss by adding both losses
8. Compute the gradient of total discriminator loss and update the discriminator optimizer parameters.

### Steps for Training Generator
1. Generate latent vector from normal distribution for mini-batch.
2. Generate fake image by feeding latent vector to generator.
3. Infer the discriminator with fake image.
4. Calculate loss for generator.
5. Compute the gradient of generator loss and update its optimizer parameters.

In [ ]:
# cell-8
# Your code here

for epoch in range(num_epochs):
    d_epoch_loss = 0
    g_epoch_loss = 0
    for i, (images, _) in enumerate(trainloader):
        batch_size = images.size(0)
        images = images.to(device)
        
        # Create labels for training discriminator and generator
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(?, ?).to(?)
        
        # ====================================================================
        # TRAINING DISCRIMINATOR
        # ====================================================================
        # Step1: Infer the discriminator on real images
        outputs = ?
        
        # Step2: Calculate the loss with real labels
        d_real_data_loss = criterion(?, ?)
        real_score = outputs
        
        # Step3: Generate random latent vectors from normal distribution
        z = torch.randn(batch_size, ?).to(?) 
        
        # Step4: Generate fake images by passing latent vector to generator
        fake_images = ?
        
        # Step5: Infer the discriminator on fake images
        outputs = ?
        
        # Step6: Calculate the loss with fake labels
        d_fake_data_loss = criterion(?, ?)
        fake_score = outputs
        
        # Step7: Calculate the total loss
        d_loss = ?
        d_epoch_loss += d_loss.item()
        
        # Step8: Update the weights
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()
        
        
        # ====================================================================
        # TRAINING GENERATOR
        # ====================================================================
        
        # Step1: Generate random latent vectors from normal distribution
        z = ?
        
        # Step2: Generate fake images by passing latent vector to generator
        fake_images = ?
        
        # Step3: Infer the discriminator on fake images
        outputs = ?
        
        # Step4: Calculate the loss with real labels
        g_loss = criterion(?, ?)
        g_epoch_loss += g_loss.item()
        
        # Step5: Update the weights
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()
        
        if (i+1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, D(x): {real_score.mean().item():.2f}, D(G(z)): {fake_score.mean().item():.2f}')

    generator_loss.append(g_epoch_loss / i)
    discriminator_loss.append(d_epoch_loss / i)
    # Visualize the intermediate results
    if (epoch+1) % 10 == 0:
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        fake_images = fake_images.detach().cpu().numpy()
        
        fig, ax = plt.subplots(1, 10, figsize=(10, 1))
        for i in range(10):
            ax[i].imshow(fake_images[i, 0], cmap='gray')
            ax[i].axis('off')
        plt.show()


### Saving Models

In [ ]:
# cell-9
torch.save(generator, 'generator.pt')
torch.save(discriminator, 'discriminator.pt')

### Plotting Learning Curves

In [ ]:
# cell-10
import matplotlib.pyplot as plt

In [ ]:
# cell-11

plt.figure(figsize=(15, 5))

# Subplot for discriminator and generator loss
plt.subplot(1, 2, 1)
plt.title("Loss Plot")
train_num_epoch = [i + 1 for i in range(len(discriminator_loss))]
plt.plot(train_num_epoch, discriminator_loss)
plt.xlabel("#Epochs")
plt.ylabel("Discriminator Training Loss")

plt.subplot(1, 2, 2)
plt.title("Loss Plot")
train_num_epoch = [i + 1 for i in range(len(discriminator_loss))]
plt.plot(train_num_epoch, generator_loss)
plt.xlabel("#Epochs")
plt.ylabel("Generator Training Loss")
plt.show()

### Generating Images by Inferencing Trained GAN

* For inferencing, we need only Generator Network.

* First we need to create a latent vector of normal distribution to feed it to generator.

In [ ]:
# cell-12
# Your code here

# Load the model
generator = ?

# Set the generator to evaluation mode
?

# Generate latent vectors 
num_samples = 10
input_noise = ? 

# Generate image using latent vectors
generated_samples = ?

generated_samples_np = generated_samples.detach().cpu().numpy()

# Visualize the generated samples
fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
for i in range(num_samples):
    axes[i].imshow(generated_samples_np[i, 0], cmap='gray')
    axes[i].axis('off')
plt.show()

### Explore Different GANs 
1. DCGANs
2. Condition GANs
3. Wasserstein GANs
4. Progressive GANs
5. StyleGANs
6. CycleGANs
7. StarGANs